<a href="https://colab.research.google.com/github/JakeFlasher/NLP-new/blob/master/Toxic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span style="color:#cc6699">  Toxic Spans Detection </span>

<span style="color:#cc6699"> SemEval competition</span>

## Libraries

In [ ]:
!pip install transformers
!/opt/bin/nvidia-smi
!pip install fsspec

In [ ]:
#library to check if all is run good in for_loop
from tqdm.notebook import tqdm

from ast import literal_eval

import json

# Data manipulation/analysis
import numpy as np
import pandas as pd
import pickle


# Setting random seed
seed = 123

# Data partitioning
from sklearn.model_selection import train_test_split

# Text preprocessing/analysis
import re
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

import spacy
import en_core_web_sm

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning models
from sklearn.feature_extraction import DictVectorizer
from sklearn import neighbors
from sklearn import linear_model #for logistic regression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from itertools import chain

from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model

#seeds
from numpy.random import seed
seed(1)
# tensorflow.random.set_random_seed(1)

## Dataset

In [ ]:
# Datasets Loading
# ind = 2
# train_url = 'https://raw.githubusercontent.com/JakeFlasher/NLP-new/master/data/train_' + str(ind) + '.csv'
# print(train_url)
# test_url = 'https://raw.githubusercontent.com/JakeFlasher/NLP-new/master/data/fold_' + str(ind) + '.csv'
trial_url = 'https://raw.githubusercontent.com/ipavlopoulos/toxic_spans/master/data/tsd_trial.csv'
train_url = 'https://raw.githubusercontent.com/ipavlopoulos/toxic_spans/master/data/tsd_train.csv'
test_url = 'https://raw.githubusercontent.com/ipavlopoulos/toxic_spans/master/data/tsd_test.csv'
# training data

dataset = pd.read_csv(train_url)
# development and validation data
tsd = pd.read_csv(trial_url) 

Example: "This is a stupid example, so thank you for nothing a!@#!@."

It comprises two toxic spans, <span style="color:Red"> "stupid"</span> and <span style="color:Blue">"a!@#!@"</span>, which have character offsets from 10 to 15 (counting starts from 0) and from 51 to 56 respectively. Systems are then expected to return the following list for this text:

[<span style="color:Red"> 10,11,12,13,14,15,</span> <span style="color:Blue"> 51,52,53,54,55,56</span>]

In [ ]:
# if we load a cell that it has list, it will need to
# build the list,otherwise it read as series of characters
dataset.spans = dataset.spans.apply(literal_eval)
dataset.head(1)

In [ ]:
# if we load a cell that it has list, it will need to
# build the list,otherwise it read as series of characters
tsd.spans = tsd.spans.apply(literal_eval)
tsd.head(1)

# Exploratory text analysis 

In [ ]:
# check if we have null data to remove-dropna
dataset.isnull().sum()

In [ ]:
# extract the the toxic word from training set

all_bad_word = []
for text in range(0,len(dataset['text'])):
    count = 0 #count how many potition are continuous
    
    bad_word = []
    for i in range(0, len(dataset.spans[text])):
        bad_word.append(dataset.text[text][dataset.spans[text][i]])
    all_bad_word.append(''.join(bad_word))

In [ ]:
all_bad_word

We can observe that we have concat word without space, for instanse, 'honkeyidiotsdimwitted' = 'honky idiots dimwitted'. Futhermore, we have f**k = fuck and sentences with bad meaning only if we can see as sentence 'three black men in America is a felon'

In [ ]:
# find all Punctuation in words  
import string 

for sentence in all_bad_word: 
    for i in sentence: 
        # checking whether the char is punctuation. 
        if i in string.punctuation: 
          
            # Printing the punctuation values  
            print("Punctuation: " + sentence) 

We can see that punctuation like `*`, `---repeat`, `/`, `$` and `'someting'` play key role, the others are in sentence:

For instance:
`s%#cks`-> keep secret the toxic word

`foola$$ `-> keep secret the toxic word

`fu/kis`-> keep secret the toxic word

`idiotF-----g` -> keep secret the toxic word

`f*** you` -> keep secret the toxic word

`'cool'` -> reveal ironic

We observe the repeat punctuation hide toxic words

Moreover, we can see that hastags hide text(eg `#yourwhitetearsarentsacredthislandis`)

In [ ]:
count_non_toxic = 0
for element in all_bad_word:
    if element == '' :
        count_non_toxic +=1
        
print('How many text not have toxic words: ' + str(count_non_toxic))

from https://towardsdatascience.com/exploratory-text-analysis-in-python-8cf42b758d9e

In [ ]:
# Prepare training corpus into one giant string
train_string = " ".join(dataset.text)
print(f"***** Extract of a train string ***** \n{train_string[:200]}", "\n")

In [ ]:
# Split train_corpus by white space
splits = train_string.split()  
print(f"***** Extract of splits ***** \n{splits[:20]}\n")

In [ ]:
print(f"Number of strings: {len(splits)}")
print(f"Number of unique strings: {len(set(splits))}")

There are approximately 300.000 strings in the training corpus with around 40.000 unique strings. 

In [ ]:
#frequency distribution for each string
freq_splits = FreqDist(splits)
print(f"***** 10 most common strings ***** \n{freq_splits.most_common(10)}", "\n")

It is obvious that the most common strings are stop words

In [ ]:
# we find all the words that they have less than 4 characters (small words)
short = set(s for s in splits if len(s)<4)
short = [(s, freq_splits[s]) for s in short]
short.sort(key=lambda x:x[1], reverse=True)
short

Many short strings appear to be stop words (like `or`) but there are also numbers(like `2`) and other short words(like `ass`).

Because we haven’t tokenised yet, some strings currently contain punctuation that is attached to a word (like `it.` or `"I`). As a result, otherwise same words are considered as different like `it`,`It`, `it?`, `it.`. Τhey probably need to normalize.

There are numbers in different forms like `('70s', 2)`,`('97%', 2)`,`('1.6', 2)` and `('$2', 2),`. Τhey probably do not need in our case, because a number is not toxic word.

In [ ]:
# we find all the words that they have more than 15 characters (longest words)
long = set(s for s in splits if len(s)>15)
long = [(s, freq_splits[s]) for s in long]
long.sort(key=lambda x:x[1], reverse=True)
long

The frequency of long string looks much lower than short strings. 

For instance `('the', 11379)` vs `('responsibilities', 3)`

There are hyphenated words: `Socio-Communists`, `we-are-shareholders`, `non-contributors` and `discrimination--except`(this one with double hyphens). If we tokenise on white space or punctuation, these strings will be split into separate words. For most cases, this will conserve the gist of the sentence. If we keep hyphenated words as they are, they won’t be as common and consequently removed as rare words.

There are websites like: `https://www.youtube.com/watch?v=bi2QKY3zW8Q`

(Extra: we do not have emails and the `@` not means email)

There are outlaw words that repeats the same character more than twice: `MOOOOOOOOOOOOOOOOOOOOOOooooooooooooooooooooooooooooo...`, `sucks!!........Hmmmm`, `HAAHAHAHAHHAAHHAHAHHAH`, `Waipahoooooooooo????` and  `succcccccccccccccckkkkkk`. If you know the correct term for these elongated words, I would love to find out. Until then, we will refer them as ‘outlaw words’. These outlaw cases seem to appear very rarely.

In [ ]:
#from https://stackoverflow.com/questions/42329766/python-nlp-british-english-vs-american-english

# method that convert the British to American English
def replace_all(text, mydict):
    for gb, us in mydict.items():
        text = text.replace(us, gb)
    return text

BvsA = json.load(open("bvsa.txt"))

In [ ]:
us_text = replace_all(train_string, BvsA)

In [ ]:
# Check if we have NLP British English and American English
if train_string == us_text:
    print(" equals " )
else:
    print( " Not equal, so we have British English and American English.\n"+
           " As a result, we may keep the American form (convert British to American English).")

If we decide to keep the covert edition, it will need to change the positions of `spans`.

In [ ]:
def summarise(pattern, strings, freq):
    """Summarise strings matching a pattern."""
    # Find matches
    compiled_pattern = re.compile(pattern)
    matches = [s for s in strings if compiled_pattern.search(s)]
    
    # Print volume and proportion of matches
    print("{} strings, that is {:.2%} of total".format(len(matches), len(matches)/ len(strings)))
    
    # Create list of tuples containing matches and their frequency
    output = [(s, freq[s]) for s in set(matches)]
    output.sort(key=lambda x:x[1], reverse=True)
    
    return output

In [ ]:
# count the frequent of numbers
summarise(r"\d", splits, freq_splits)

It is frequent to have numbers (54%)

We have a variety of numbers, percentage, numeric, money, date and others. But the numbers, as we said before, do not reveal toxicity. We may need to drop numbers when tokenising. 

In [ ]:
# find the frequent of hyphenated words
summarise(r"\w+-+\w+", splits, freq_splits)

It is frequent (44%), these ones definitely need to be separated.

From glancing through the hyphenated words, it makes more sense to separate them out to keep the data simple. For example: We should tokenise ‘same-sex’ to 2 tokens: `[‘same’, ‘sex’]` instead of 1 token: `[‘same-sex’]`. 

In [ ]:
#count requent of words combined by other punctuation
summarise(r"\w+[_!&/)(<\|}{\[\]]\w+", splits, freq_splits)

Not too frequent, but it is important.

We can observe that some toxic word hidden behind the punctuation. 

For instance, `'(D)onkeys'`->`'Donkeys'`, `fu/k`->`fuck`

Furthermore, we have same similars words, which it seems to be unuseful, like `is/was`,`he/she` or `'him/her'`. In addition, we have same words in different series, like `is/was` and `was/is`

In [ ]:
# count frequent of outlaw words
def find_outlaw(word):
    """Find words that contain a same character 3+ times in a row."""
    is_outlaw = False
    for i, letter in enumerate(word):
        if i > 1:
            if word[i] == word[i-1] == word[i-2] and word[i].isalpha():
                is_outlaw = True
                break
    return is_outlaw

outlaws = [s for s in splits if find_outlaw(s)]
print("{} strings, that is {:.2%} of total".format(len(outlaws), len(outlaws)/ len(splits)))
outlaw_freq = [(s, freq_splits[s]) for s in set(outlaws)]
outlaw_freq.sort(key=lambda x:x[1], reverse=True)
outlaw_freq

Not too frequent, these may not worth correcting for because there too few cases. However, we will need to seperate, because it hidde toxic words like `killls`

In [ ]:
# tokenise the text into alphabetic tokens
tokeniser = RegexpTokenizer("[A-Za-z]+")
tokens = tokeniser.tokenize(train_string)
print(tokens[:20], "\n")

In [ ]:
print(f"Number of tokens: {len(tokens)}")
print(f"Number of unique tokens: {len(set(tokens))}")

There are 300.000 tokens in the training data with around 23.000 unique tokens.

In [ ]:
# convert all tokens into lowercase and lemmatise them
nltk.download('wordnet')
lemmatiser = WordNetLemmatizer()
tokens_norm = [lemmatiser.lemmatize(t.lower(), "v") for t in tokens]
print(f"Number of unique tokens: {len(set(tokens_norm))}")

In [ ]:
temp = round(1 - len(set(tokens_norm))/len(set(tokens)), 2)
print('We drop by about ' + str(temp* 100) + '%')

**!caution! If we keep the lemmatise method, we should change the `spans`**

In [ ]:
# Create list of token lengths for each token
token_length = [len(t) for t in tokens]

# Average number of characters per token
print(f"Average number of characters per token: {round(np.mean(token_length),4)}")

In [ ]:
sns.set(style="whitegrid")
ax = sns.violinplot(token_length)

In [ ]:
token_length_2 = [len(t) for t in tokens if len(t)<15]

sns.set(style="whitegrid")
ax = sns.violinplot(token_length_2)

In [ ]:
# Plot distribution
plt.figure(figsize=(12, 12))
sns.countplot(y=token_length)
plt.title("Counts of token length", size=20);

There are a few tokens that is very long but also very rare.

In [ ]:
# load stopwords and add some more
nltk.download('stopwords')
stop_words = stopwords.words("english")
stop_words.extend(["cannot", "could", "done", "let", "may" "mayn",  "might", "must", "need", "ought", "oughtn", "shall", "would", "br"])

In [ ]:
# the most common stop words in our corpus
freq_stopwords = [(sw, tokens_norm.count(sw)) for sw in stop_words]
freq_stopwords.sort(key=lambda x: x[1], reverse=True)
freq_stopwords[:10]

In [ ]:
# count all the stopwords
n_stopwords = len([t for t in tokens_norm if t in stop_words])
print(f"{n_stopwords} tokens are stop words.")
print(f"The proportion of tokens are stop words is: {round(100*n_stopwords/len(tokens_norm),2)}%.")

About half of the tokens are stop words

In [ ]:
# remove stop words
tokens_clean = [t for t in tokens_norm if t not in stop_words]
print(f"Number of tokens: {len(tokens_clean)}")

In [ ]:
def preprocess_text(text):
    """Preprocess text into normalised tokens."""
    # Tokenise words into alphabetic tokens
    tokeniser = RegexpTokenizer(r'[A-Za-z]{2,}')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stop_words]
    return keywords

def get_frequent_ngram(corpus, ngram, n=20):
    """Find most common n n-grams tokens."""
    # Preprocess each document
    documents = [preprocess_text(document) for document in corpus]
    
    # Find ngrams per document
    n_grams = [list(ngrams(document, ngram)) for document in documents]
    
    # Find frequency of ngrams
    n_grams_flattened = [item for sublist in n_grams for item in sublist]
    freq_dist = FreqDist(n_grams_flattened)
    top_freq = freq_dist.most_common(n)
    return pd.DataFrame(top_freq, columns=["ngram", "count"])

In [ ]:
# Get frequent ngrams for all 4
for i in range(1,5):
    mapping = {1:"uni", 2:"bi", 3:"tri", 4:"four"}
    plt.figure(figsize=(12,10))
    sns.barplot(x="count", y="ngram", data=get_frequent_ngram(dataset['text'], i))
    plt.title(f"Most common {mapping[i]}grams");

The word `trump` (logically from `trumpet` after lemm)and `stupid` looks quite frequent compared to the other frequent words.

In [ ]:
def get_avg_word_len(x):
    words = x.split()
    word_len = 0
    for word in words:
        word_len = word_len + len(word)
        
    return word_len/len(words)


In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
# tokeniser = RegexpTokenizer("[A-Za-z]+")
new_dataset = dataset.copy()

#delete the  None (NA) types in your DF
new_dataset.dropna(inplace=True)

new_dataset["n_sentences"] = new_dataset["text"].apply(sent_tokenize).apply(len)
new_dataset["tokens"] = new_dataset["text"].apply(tokeniser.tokenize)
new_dataset["n_tokens"] = new_dataset["tokens"].apply(len)
new_dataset["n_characters"] = dataset["text"].apply(len)
new_dataset["n_stopwords"] = new_dataset["tokens"].apply(lambda tokens: len([t for t in tokens if t in stop_words]))
new_dataset["p_stopwords"] = new_dataset["n_stopwords"]/new_dataset["n_tokens"]

# Inspect head
new_dataset.head()

In [ ]:
sns.distplot(new_dataset['n_tokens'],bins=100)

In [ ]:
sns.distplot(new_dataset['n_characters'],bins=100)

In [ ]:
sns.pairplot(new_dataset)

In [ ]:
new_dataset.describe()

In [ ]:
bad_word_train = pd.DataFrame(all_bad_word, columns = ['toxic'])
bad_word_train

In [ ]:
#delete the  None (NA) types in your DF
bad_word_train.replace("",float("NaN") , inplace=True)
bad_word_train = bad_word_train.dropna()

bad_word_train

In [ ]:
temp = ' '.join(bad_word_train.toxic.tolist())
bad_word_train_clean = preprocess_text(temp) #call the function

In [ ]:
# freguent of toxic words in corpus
bad_word_train_freq = FreqDist(bad_word_train_clean)

In [ ]:
# keep the 50 most common words
common_bad_word_train_freq = bad_word_train_freq.most_common(50)

In [ ]:
#FreqDist to Pandas dataframe
df_bad_word_train_freq= pd.DataFrame(common_bad_word_train_freq, 
                                    columns = ['Term','Frequency'])
df_bad_word_train_freq.head()

In [ ]:
plt.figure(figsize=(12,10))
sns.barplot(x="Frequency", y="Term", data=df_bad_word_train_freq)
plt.title("Top toxic words in corpus");

# Ιnformation Extraction

In [ ]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

In [ ]:
def tranform_dataset(dataset):
    list_doc = [] # is the number of each row in dataset
    list_tokens = [] # token of each text
    list_span_start = [] # start position of token
    list_span_label = [] # label initialize

    # tranform the dataset as text to token
    for text_i in tqdm(range(0,len(dataset['text']))):

        list_doc.append(text_i)
        
        # tokenization by space " "
        token = dataset.text[text_i].split()
        list_tokens.append(token)

        # code to find the start position of each word
        start_pos = []
        count = 0
        for i in token:
            start_pos.append(count)
            count+=len(i)+1
        list_span_start.append(start_pos)
        
        list_span_label.append([0]* len(start_pos))
        
    
    # add the new list into a dataframe
    df_token = pd.DataFrame(list_doc, columns=['doc'])
    df_token['token'] = list_tokens
    df_token['span_start'] = list_span_start
    
    df_token['toxic'] = list_span_label
    
    return df_token

In [ ]:
df_token = tranform_dataset(dataset)

In [ ]:
dataset

In [ ]:
df_token

In [ ]:
def detect_labels(df_token,dataset):   
    # find toxic tokens
    for i in tqdm(range(0,len(df_token.token))):

        #eg span_dataset = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...
        span_dataset = dataset.spans[i]
        all_dataset = df_token.span_start[i]

        pos = 0
        for x in all_dataset:
            if x in span_dataset:
                df_token.toxic[i][pos]=1
            pos+=1
           
    return df_token

In [ ]:
df = detect_labels(df_token,dataset)

In [ ]:
df

In [ ]:
df.iloc[1]

In [ ]:
# where to save it, usually as a .pkl
df.to_pickle('myDataset_training')  

**Same code for develop/validation dataset `tsd`**

In [ ]:
df_dev = tranform_dataset(tsd)

In [ ]:
df_dev = detect_labels(df_dev,tsd)

In [ ]:
df_dev

In [ ]:
for i in range(len(df_dev.doc)):
    if df_dev.doc[i]==8:
        print(df_dev.toxic[i])
        print(df_dev.token[i])

In [ ]:
# where to save it, usually as a .pkl
df_dev.to_pickle('myDataset_develop') 

# Text Cleaning

In [ ]:
def clean(df_train_clean): 

    #lowercase
    df_train_clean['token'] = df_train_clean['token'].str.lower()
    
    #Fisrt and foremost we delete \n , because we have \nDon't
    df_train_clean['token'].replace('\n', '' , inplace=True)
    
    #remove `?`
    df_train_clean['token'].replace(re.compile(r'[?]'), '', inplace=True)

    # remove specific char
    df_train_clean['token'].replace(re.compile(r'[)"(,:‘}{=><]'), '', inplace=True) #correct
    
    # tranform percentage to TAG (eg 34% )
    df_train_clean['token'].replace(re.compile(r'\d+[%]'), 'NUM', inplace=True)
    
    # tranform money to TAG (eg 34$ )
    df_train_clean['token'].replace(re.compile(r'\d+[$]'), 'MONEY', inplace=True)
    
    # tranform money to TAG (eg $34)
    df_train_clean['token'].replace(re.compile(r'[$]\d+'), 'MONEY', inplace=True)
    
    # tranform money to TAG (eg $34)
    df_train_clean['token'].replace(re.compile(r'[$]\w+'), 'TOXIC', inplace=True)
    
    # tranform multi full stop to one full stop (eg MAm....... )
    df_train_clean['token'].replace(re.compile(r'[.]+'), '.', inplace=True)
    
    #remove `.`
    df_train_clean['token'].replace(re.compile(r'[.]'), '', inplace=True)
    
    # change hide toxic words (e.g f@@k = f**k)
    df_train_clean['token'].replace(re.compile(r"(\w+[*+$#@&%!]+\w+)"), 'TOXIC', inplace=True)
    
    # change hide toxic words (e.g f***)
    df_train_clean['token'].replace(re.compile(r"(\w+[*$#@&%])"), 'TOXIC', inplace=True)
    
    #remove `.`
    df_train_clean['token'].replace(re.compile(r'[!]'), '', inplace=True)
    
    # tranform reference to TAG
    df_train_clean['token'].replace(re.compile(r'@(\w+)'), 'REF' , inplace=True)
    
    #Remove repeating characters from words (eg. my loveeeee)
    df_train_clean['token'].replace(to_replace=r'(.)\1\1+',  value=r'\1', regex=True, inplace=True)
    

    # tranform urls to TAG   #help for: https://www.geeksforgeeks.org/python-check-url-string/
    df_train_clean['token'].replace(re.compile(r'(https?://[^\s]+)'), 'URL' , inplace=True)    


    #remove emoji
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    df_train_clean['token'].replace(emoji_pattern, 'EMO' , inplace=True)




    return df_train_clean['token']

In [ ]:
#clean dataset code

df_train_clean = df.copy()
for doc in tqdm(range(len(df_train_clean.token))):
    temp = pd.DataFrame(df_train_clean.token[doc],columns=['token'])
    df_train_clean.token[doc] = clean(temp).values.tolist()
    
df_develop_clean = df_dev.copy()
for doc in tqdm(range(len(df_develop_clean.token))):
    temp = pd.DataFrame(df_develop_clean.token[doc],columns=['token'])
    df_develop_clean.token[doc] = clean(temp).values.tolist()

In [ ]:
df_train_clean.head()

# System 1 (with LSTM)

## Data Pre-processing for Neural Network

In [ ]:
def flatten(input):
    new_list = []
    new_doc = []
    count_doc = 0
    for i in input:
        for j in i:
            new_list.append(j)
            new_doc.append(str(count_doc))
        count_doc+=1
    return new_list, new_doc

In [ ]:
flat,doc_ids = flatten(df_train_clean.token.values) 
labels,doc_ids = flatten(df_train_clean.toxic.values)

df_train = pd.DataFrame(flat, columns= ['token'])
df_train['toxic'] = labels
df_train['doc'] = doc_ids
df_train

In [ ]:
flat,doc_ids = flatten(df_develop_clean.token.values)
labels,doc_ids = flatten(df_develop_clean.toxic.values)

df_develop = pd.DataFrame(flat, columns= ['token'])
df_develop['toxic'] = labels
df_develop['doc'] = doc_ids
df_develop

In [ ]:
for i in range(len(df_develop.doc)):
    if df_develop.doc[i]=='3':
        print(df_develop.token[i])

---

from https://github.com/ferrygun/NERC

In [ ]:
words = list(set(df_train["token"].values))
words.append("UNK") #georg
words.append("ENDPAD")

words_val = list(set(df_develop["token"].values))
words_val.append("ENDPAD")

In [ ]:
n_words = len(words)
print('Number of words(=tokens)in training dataset: '+str(n_words))

n_words_val = len(words_val)
print('Number of words(=tokens) in validation dataset: '+str(n_words_val))

In [ ]:
tags = list(set(df_train["toxic"].values))
n_tags = len(tags)
print('Number of tags : '+str(n_tags)+ ' = ' +str(tags))

## Features Extraction on Sentences

In preparation for features extraction, we create the following helpers:

`word2idx`: convert word to index

 `tag2idx`: convert tag to index
 
 `idx2word`: convert index to word
 
 `idx2tag`: convert index to tag

In [ ]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

idx2word = {i: w for w, i in word2idx.items()}
idx2tag = {i: w for w, i in tag2idx.items()}

In [ ]:
len(word2idx)

**Solve the problem out of vocabulary**

In [ ]:
df_develop_old = df_develop.copy()

for w in tqdm(words_val):
    if w not in word2idx:
        df_develop.token.replace(w,'UNK', inplace=True)

In [ ]:
# print new alternative sentence
for i in range(len(df_develop.doc)):
    if df_develop.doc[i]=='3':
        print(df_develop.token[i])

We write SentenceGetter class to retrieve sentences with their words, POS and tags from the datasets. The code is taken from https://www.depends-on-the-definition.com/named-entity-recognition-conditional-random-fields-python/

In [ ]:
def mySentenceGetter(data):
    doc_id = 0
    sentences = []
    temp_doc = []
    for i in tqdm(range(len(data.doc))):
        if data.doc[i]==doc_id :
            temp_doc.append((data["token"][i], data["toxic"][i]))
        else:
            sentences.append(temp_doc)
            doc_id = data.doc[i]
            temp_doc = []
            temp_doc.append((data["token"][i], data["toxic"][i]))
    sentences.append(temp_doc)
    return sentences[1:]    

In [ ]:
sentences = mySentenceGetter(df_train)

sentences_val = mySentenceGetter(df_develop)

We need to convert the sentences to numbers as Keras work better:

In [ ]:
X_train = [[word2idx[w[0]] for w in s] for s in sentences]

In [ ]:
X_test_prod = [[word2idx[w[0]] for w in s] for s in sentences_val]

In [ ]:
print(X_train[0])

In [ ]:
for i in X_train[0]:
    print(idx2word[i],end= " ") 

In [ ]:
for i in X_test_prod[0]:
    print(idx2word[i],end= " ") 

In [ ]:
print(X_test_prod[0])

In [ ]:
idx2word[n_words-1]

Padding: The LSTM layers accept sequences of same length only. Therefore we will want to transform our list of token_sequences which is lists of integers into a matrix of shape (token_sequences, max_len). We can use any length as max_len. In this project we will be using length of the longest sequence as max_len. The sequences that are shorter than max_len are padded with a specified value at the end.

We padding the sentences to the same length (max_len) and fill with index value of “ENDPAD” or 23614.

In [ ]:
# find longest sentence
max_len = max([len(s) for s in idx2word.values()])
max_len

In [ ]:
X_train = pad_sequences(maxlen=max_len, sequences=X_train, padding="post",
                  value=n_words - 1)

X_test_prod = pad_sequences(maxlen=max_len, sequences=X_test_prod, padding="post",
                  value=n_words - 1)

In [ ]:
print('Result after padding')
X_test_prod[4]

In [ ]:
for k in X_test_prod[4]:    
    print(idx2word[k],end= " ") 

#### Labels
Convert labels (tags) to numbers

Padding the labels (we padding the Labels to the same length (max_len) and fill with index value of entity `non` or `0`.)

In [ ]:
y_train = [[w[1] for w in s] for s in sentences]

y_test_prod = [[w[1] for w in s] for s in sentences_val]

In [ ]:
y_train = pad_sequences(maxlen=max_len, sequences=y_train, 
                  padding="post", value=2)###value=0 ->value=2

y_test_prod = pad_sequences(maxlen=max_len, sequences=y_test_prod, 
                  padding="post", value=2)###value=0->value=2

In [ ]:
y_train[0]

In [ ]:
tag2idx['ENDPAD']= 2###
idx2tag[2]= 'ENDPAD'###

In [ ]:
#update number of classes
n_tags = len(tag2idx)

Perform One-Hot Encoding for labels

In [ ]:
y_train = [to_categorical(i, num_classes=n_tags) for i in y_train] 

y_test_prod = [to_categorical(i, num_classes=n_tags) for i in y_test_prod]

In [ ]:
y_train[0].shape

## Neural Network Model (LSTM)

We create and compile the sequential model with the following architecture.

RNNs are capable of handling different input and output combinations. We will use many to many architectures for this ask. Refer to the last architecture in the image given below. Our task is to output tag (y) for a token (X) ingested at each time step.

![title](images/rnn.png)

form from https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

We are building a simple model with 3 layers.

- **Layer 1 - Embedding layer** : We will feed the padded sequences of equal lengthto the embedding layer. Once the network has been trained, each token will get transformed into a vector of n dimensions. We have chosen the n dimensions to be. 


- **Layer 2 - Bidirectional LSTM** : Bidirectional lstm takes a recurrent layer (e.g. the first LSTM layer) as an argument. This layer takes the output from the previous embedding layer. It also allows you to specify the merge mode, that is how the forward and backward outputs should be combined before being passed on to the next layer. The default mode is to concatenate, where the outputs are concatenated together, providing double the number of outputs to the next layer.


- **Layer  LSTM Layer** (at end we do not put): An LSTM network is a recurrent neural network that has LSTM cell blocks in place of our standard neural network layers. These cells have various components called the input gate, the forget gate and the output gate. (at end we do not put)



- **Layer 3- TimeDistributed  Layer** : We are dealing with Many to Many RNN Architecture where we expect output from every input sequence for example (a1 →b1, a2 →b2… an →bn) where a and b are inputs and outputs of every sequence. The TimeDistributeDense layers allow you to apply Dense(fully-connected) operation across every output over every time-step. If you don't use this, you would only have one final output.

In [132]:
input = Input(shape=(max_len,))

# Add Embedding layer
model = Embedding(input_dim=n_words, output_dim=50, 
                  input_length=max_len, mask_zero=True)(input)

model = Dropout(0.5)(model)

# Add bidirectional LSTM
model = Bidirectional(LSTM(units=100, return_sequences=True,
                           recurrent_dropout=0.1))(model)
# Add timeDistributed Layer
out = TimeDistributed(Dense(n_tags, activation="sigmoid"))(model) ###n_tags not(n_tags+1)

In [133]:
model = Model(input, out)

In [134]:
#Optimiser 
opt = tensorflow.keras.optimizers.Adam(lr=0.01, decay=1e-6)

model.compile(optimizer=opt, loss="categorical_crossentropy",
              metrics=["accuracy"])  

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [135]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 125)]             0         
                                                                 
 embedding (Embedding)       (None, 125, 50)           248300    
                                                                 
 dropout (Dropout)           (None, 125, 50)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 125, 200)         120800    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 125, 3)           603       
 ibuted)                                                         
                                                                 
Total params: 369,703
Trainable params: 369,703
Non-trainable

In [136]:
early = EarlyStopping(monitor="val_loss", 
                              mode="min" if "loss" in "val_loss" else "max", 
                              patience=1, verbose=1,
                              min_delta=0.0001, restore_best_weights=True)

In [137]:
# training mode
history = model.fit(X_train, np.array(y_train), 
                    batch_size=32, 
                    epochs=10, 
                    validation_split=0.1,
                    verbose=1, 
                    callbacks=[early])

Epoch 1/10
20/20 [==============================] - 38s 1s/step - loss: 0.2623 - accuracy: 0.9265 - val_loss: 0.0882 - val_accuracy: 0.9759
Epoch 2/10
20/20 [==============================] - 29s 1s/step - loss: 0.0600 - accuracy: 0.9833 - val_loss: 0.0738 - val_accuracy: 0.9759
Epoch 3/10
20/20 [==============================] - 29s 1s/step - loss: 0.0525 - accuracy: 0.9836 - val_loss: 0.0709 - val_accuracy: 0.9761
Epoch 4/10
20/20 [==============================] - 29s 1s/step - loss: 0.0445 - accuracy: 0.9849 - val_loss: 0.1078 - val_accuracy: 0.9763
Epoch 00004: early stopping


#### Plot Accuracy and Loss

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

***Initialize***

In [ ]:
toxic_label = 1
not_toxic_label = 0
class_num = 2

**F1 function**

In [ ]:
def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

**Get offset**

In [ ]:
threshold = 0.18 #by user test

text_predictions = model.predict(X_test_prod)

my_pred = []
for i in tqdm(range(len(text_predictions))):
    max_prob = 0
    tmp = []
    for j in range(len(text_predictions[i])):
        #if find class pad, stop    
        if text_predictions[i][j][2]>0.8:
            break
         
        if text_predictions[i][j][1]>threshold:
            tmp.append(1)
        else:
            tmp.append(0)
            
        
    my_pred.append(tmp)
        

In [ ]:
df_temp = pd.DataFrame({'pred':my_pred})
df_temp

In [ ]:
toxic_offsets_all=[]
for i in range(len(df_dev.token)):
    toxic_offsets = []
    current_offset = 0
    for j in range(len(df_dev.token[i])):
        if df_temp.pred[i][j]==1:
            toxic_offsets.extend(list(range(current_offset, current_offset+len(df_dev.token[i][j]))))
        current_offset += len(df_dev.token[i][j]) + 1
    toxic_offsets_all.append(toxic_offsets)

In [ ]:
df_predict = pd.DataFrame({'pred':toxic_offsets_all})
df_predict 

In [ ]:
tsd

In [ ]:
tsd["predictions"] = df_predict.pred
tsd["f1_scores"] = tsd.apply(lambda row: f1(row.predictions, row.spans), axis=1)

In [ ]:
tsd

In [ ]:
from scipy.stats import sem
print (f"F1 = {tsd.f1_scores.mean():.2f} ± {sem(tsd.f1_scores):.2f}")

In [ ]:
_ = tsd.f1_scores.plot(kind="box")

In [ ]:
model.save("model.h5")

## Test set Results

In [ ]:
# test data
dataset_test = pd.read_csv("data/tsd_test.csv")

In [ ]:
dataset_test

In [ ]:
list_doc = [] # is the number of each row in dataset
list_tokens = [] # token of each text

# tranform the dataset as text to token
for text_i in tqdm(range(0,len(dataset_test['text']))):

    list_doc.append(text_i)
        
    # tokenization by space " "
    token = dataset_test.text[text_i].split()
    list_tokens.append(token)

    # add the new list into a dataframe
    df_test_start = pd.DataFrame(list_doc, columns=['doc'])
    df_test_start['token'] = list_tokens

In [ ]:
df_test_start

In [ ]:
#clean dataset code
df_test_start_clean = df_test_start.copy()
for doc in tqdm(range(len(df_test_start_clean.token))):
    temp = pd.DataFrame(df_test_start_clean.token[doc],columns=['token'])
    df_test_start_clean.token[doc] = clean(temp).values.tolist()

In [ ]:
df_test_start_clean 

In [ ]:
flat,doc_ids = flatten(df_test_start_clean.token) #clean dataset

df_test = pd.DataFrame(flat, columns= ['token'])
df_test['doc'] = doc_ids
df_test

In [ ]:
words_test = list(set(df_test["token"].values))
words_test.append("ENDPAD")

In [ ]:
for w in tqdm(words_test):
    if w not in word2idx:
        df_test.token.replace(w,'UNK', inplace=True)
        print(w)

In [ ]:
def mySentenceGetter_test(data):
    doc_id = 0
    sentences = []
    temp_doc = []
    for i in tqdm(range(len(data.doc))):
        if data.doc[i]==doc_id :
            temp_doc.append((data["token"][i]))
        else:
            sentences.append(temp_doc)
            doc_id = data.doc[i]
            temp_doc = []
            temp_doc.append((data["token"][i]))
    sentences.append(temp_doc)
    return sentences[1:] 

In [ ]:
sentences_test = mySentenceGetter_test(df_test)

In [ ]:
X_test = [[word2idx[w] for w in s] for s in sentences_test]

In [ ]:
X_test = pad_sequences(maxlen=max_len, sequences=X_test, padding="post",
                  value=n_words - 1)

In [ ]:
text_predictions = model.predict(X_test)

my_pred = []
for i in tqdm(range(len(text_predictions))):
    max_prob = 0
    tmp = []
    for j in range(len(text_predictions[i])):
        #if find class pad, stop    
        if text_predictions[i][j][2]>0.8:
            break
            
        if text_predictions[i][j][1]>threshold:
            tmp.append(1)
        else:
            tmp.append(0)
            
        
    my_pred.append(tmp)

In [ ]:
df_temp = pd.DataFrame({'pred':my_pred})
df_temp

In [ ]:
toxic_offsets_all=[]
for i in range(len(df_test_start.token)):
    toxic_offsets = []
    current_offset = 0
    for j in range(len(df_test_start.token[i])):
        if df_temp.pred[i][j]==1:
            toxic_offsets.extend(list(range(current_offset, current_offset+len(df_test_start.token[i][j]))))
        current_offset += len(df_test_start.token[i][j]) + 1
    toxic_offsets_all.append(toxic_offsets)

In [ ]:
df_predict = pd.DataFrame({'pred':toxic_offsets_all})
df_predict 

In [ ]:
predictions = df_predict.pred.to_list()
ids = df_predict.index.to_list()

# write in a prediction file named "spans-pred.txt"
with open("spans-pred.txt", "w") as out:
      for uid, text_scores in zip(ids, predictions):
            out.write(f"{str(uid)}\t{str(text_scores)}\n")

In [ ]:
! zip -r random_predictions.zip ./spans-pred.*

---

# System 2 (with LSTM-CRFs)

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [ ]:
input = Input(shape=(max_len,))
model_crf = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len)(input)  # 20-dim embedding
model_crf = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model_crf)  # variational biLSTM
model_crf = TimeDistributed(Dense(50, activation="relu"))(model_crf)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model_crf)  # output

In [ ]:
model_crf = Model(input, out)

In [ ]:
model_crf.compile(optimizer="rmsprop", loss= crf.loss_function,
              metrics=[crf.accuracy])  

In [ ]:
# training
history = model_crf.fit(X_train, np.array(y_train), 
                    batch_size=32, 
                    epochs=10, 
                    validation_split=0.1,
                    verbose=1, 
                    callbacks=[early])

In [ ]:
threshold = 0.18 #by user test

text_predictions = model_crf.predict(X_test_prod)

my_pred = []
for i in tqdm(range(len(text_predictions))):
    max_prob = 0
    tmp = []
    for j in range(len(text_predictions[i])):
        #if find class pad, stop    
        if text_predictions[i][j][2]>0.8:
            break
         
        if text_predictions[i][j][1]>threshold:
            tmp.append(1)
        else:
            tmp.append(0)
            
        
    my_pred.append(tmp)

In [ ]:
df_temp = pd.DataFrame({'pred':my_pred})
df_temp

In [ ]:
toxic_offsets_all=[]
for i in range(len(df_dev.token)):
    toxic_offsets = []
    current_offset = 0
    for j in range(len(df_dev.token[i])):
        if df_temp.pred[i][j]==1:
            toxic_offsets.extend(list(range(current_offset, current_offset+len(df_dev.token[i][j]))))
        current_offset += len(df_dev.token[i][j]) + 1
    toxic_offsets_all.append(toxic_offsets)

In [ ]:
df_predict = pd.DataFrame({'pred':toxic_offsets_all})
df_predict 

In [ ]:
tsd["predictions_crf"] = df_predict.pred
tsd["f1_scores_crf"] = tsd.apply(lambda row: f1(row.predictions_crf, row.spans), axis=1)

In [ ]:
tsd[:25]

In [ ]:
from scipy.stats import sem
print (f"F1 = {tsd.f1_scores_crf.mean():.2f} ± {sem(tsd.f1_scores_crf):.2f}")

In [ ]:
_ = tsd.f1_scores.plot(kind="box")

In [ ]:
model_crf.save("model_crf.h5")

**Join Results**

In [ ]:
count_crf =0
count=0
for i in range(len(tsd.f1_scores_crf)):

    if tsd.predictions_crf[i]==[]:

        if tsd.f1_scores_crf[i]>=tsd.f1_scores[i]:
            count_crf+=1
        else:
            count+=1
        
print(count_crf)
print(count)

In [ ]:
tsd["predictions_join"] = df_predict.pred
for i in range(len(tsd["predictions_join"])):
    if tsd.predictions_crf[i]==[] and tsd.predictions[i]==[]:
        tsd["predictions_join"][i]=[]
        next
    if tsd["predictions"][i]==[]:
        tsd["predictions_join"][i]=tsd["predictions_crf"][i]
    if tsd["predictions_crf"][i]==[]:
        tsd["predictions_join"][i]=tsd["predictions"][i]

In [ ]:
tsd["f1_scores_join"] = tsd.apply(lambda row: f1(row.predictions_join, row.spans), axis=1)

In [ ]:
print (f"F1 = {tsd.f1_scores_join.mean():.2f} ± {sem(tsd.f1_scores_join):.2f}")

---

# System 3 (with sklearn)
### Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from ast import literal_eval

In [ ]:
df_train

In [ ]:
df_develop

***Create one hot dataframe***

In [ ]:
# find toxic words after cleaning text
toxic_words_list = []
nontoxic_words_list = []
for i in tqdm(range(len(df_train.toxic))):
    if df_train.toxic[i]==1:
        toxic_words_list.append(df_train.token[i])
        print(df_train.token[i])
    else:
        nontoxic_words_list.append(df_train.token[i])

In [ ]:
len(toxic_words_list)

In [ ]:
# put toxic words in dataframe
df_fast_1= pd.DataFrame(toxic_words_list,columns=['token_toxic'])

In [ ]:
# transform to one hot
df_fast = pd.get_dummies(df_fast_1.token_toxic)

In [ ]:
df_fast

In [ ]:
df_y_1 = pd.DataFrame(1, index=np.arange(len(df_fast)),columns= ['toxic'])

In [ ]:
df_fast_2 = pd.DataFrame(0, index=np.arange(len(nontoxic_words_list)), columns=list(df_fast.columns))

In [ ]:
df_fast_2

In [ ]:
df_y_2 = pd.DataFrame(0, index=np.arange(len(df_fast_2)),columns= ['toxic'])

In [ ]:
frames = [df_fast, df_fast_2]
df_train_oneHot = pd.concat(frames)

frames = [df_y_1,df_y_2]
df_y = pd.concat(frames)

In [ ]:
df_train_oneHot

In [ ]:
y = df_y.toxic.values
classes = np.unique(y)
classes = classes.tolist()

In [ ]:
X_train = df_train_oneHot
y_train = y

**Models from sklearn**

In [ ]:
sgd = SGDClassifier()
sgd.partial_fit(X_train, y_train, classes)

In [ ]:
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

In [ ]:
pa =PassiveAggressiveClassifier()
pa.partial_fit(X_train, y_train, classes)

**Preprocessing**

In [ ]:
words = list(set(df_train["token"].values))
word2idx = {w: i for i, w in enumerate(words)}

words_val = list(set(df_develop["token"].values))

In [ ]:
df_develop_old = df_develop.copy()

for w in tqdm(words_val):
    if w not in word2idx:
        df_develop.token.replace(w,'UNK', inplace=True) #georg
#         #find the most similar words for those that are out of vocabulary
#         similar = ft_model.wv.most_similar(positive=[w], topn=1)
#         df_develop.token.replace(w,similar[0][0], inplace=True)

In [ ]:
df_develop['temp_toxic']= [0]*len(df_develop.token)
df_develop

In [ ]:
for i in tqdm(range(len(df_develop.token))):
    if df_develop.token[i] in toxic_words_list:
         df_develop.temp_toxic[i]=1

In [ ]:
df_dev_fast = pd.DataFrame(0, index=np.arange(len(df_develop.token)), columns=list(df_fast.columns))

In [ ]:
for i in tqdm(range(len(df_develop.token))):
    if df_develop.temp_toxic[i]==1:
        df_dev_fast[df_develop.token[i]][i]=1

**Predict for Naive Bayes Classifier**

In [ ]:
y_pred=nb.predict(df_dev_fast)

In [ ]:
doc_id = 0
sentences = []
temp_doc = []
for i in tqdm(range(len(df_develop.doc))):
    if df_develop.doc[i]==doc_id :
        temp_doc.append(y_pred[i])
    else:
        sentences.append(temp_doc)
        doc_id = df_develop.doc[i]
        temp_doc = []
        temp_doc.append(y_pred[i])
sentences.append(temp_doc)

In [ ]:
df_temp = pd.DataFrame({'pred':sentences[1:]})
df_temp

In [ ]:
df_dev

In [ ]:
tsd = pd.read_csv("data/tsd_trial.csv") 
tsd.spans = tsd.spans.apply(literal_eval)
tsd.head(1)

In [ ]:
toxic_offsets_all=[]
for i in range(len(df_dev.token)):
    toxic_offsets = []
    current_offset = 0
    for j in range(len(df_dev.token[i])):
        if df_temp.pred[i][j]==1:
            toxic_offsets.extend(list(range(current_offset, current_offset+len(df_dev.token[i][j]))))
        current_offset += len(df_dev.token[i][j]) + 1
    toxic_offsets_all.append(toxic_offsets)

In [ ]:
df_predict = pd.DataFrame({'pred':toxic_offsets_all})
df_predict

In [ ]:
tsd["predictions"] = df_predict.pred
tsd["f1_scores"] = tsd.apply(lambda row: f1(row.predictions, row.spans), axis=1)

In [ ]:
from scipy.stats import sem
print (f"F1 = {tsd.f1_scores.mean():.2f} ± {sem(tsd.f1_scores):.2f}")

**Predict for PassiveAggressive Classifier**

In [ ]:
y_pred=pa.predict(df_dev_fast)

In [ ]:
doc_id = 0
sentences = []
temp_doc = []
for i in tqdm(range(len(df_develop.doc))):
    if df_develop.doc[i]==doc_id :
        temp_doc.append(y_pred[i])
    else:
        sentences.append(temp_doc)
        doc_id = df_develop.doc[i]
        temp_doc = []
        temp_doc.append(y_pred[i])
sentences.append(temp_doc)
df_temp = pd.DataFrame({'pred':sentences[1:]})

In [ ]:
tsd = pd.read_csv("data/tsd_trial.csv") 
tsd.spans = tsd.spans.apply(literal_eval)

toxic_offsets_all=[]
for i in range(len(df_dev.token)):
    toxic_offsets = []
    current_offset = 0
    for j in range(len(df_dev.token[i])):
        if df_temp.pred[i][j]==1:
            toxic_offsets.extend(list(range(current_offset, current_offset+len(df_dev.token[i][j]))))
        current_offset += len(df_dev.token[i][j]) + 1
    toxic_offsets_all.append(toxic_offsets)
df_predict = pd.DataFrame({'pred':toxic_offsets_all})

In [ ]:
tsd["predictions"] = df_predict.pred
tsd["f1_scores"] = tsd.apply(lambda row: f1(row.predictions, row.spans), axis=1)
print (f"F1 = {tsd.f1_scores.mean():.2f} ± {sem(tsd.f1_scores):.2f}")

**Predict for SGD Classifier**

In [ ]:
y_pred=sgd.predict(df_dev_fast)

In [ ]:
doc_id = 0
sentences = []
temp_doc = []
for i in tqdm(range(len(df_develop.doc))):
    if df_develop.doc[i]==doc_id :
        temp_doc.append(y_pred[i])
    else:
        sentences.append(temp_doc)
        doc_id = df_develop.doc[i]
        temp_doc = []
        temp_doc.append(y_pred[i])
sentences.append(temp_doc)
df_temp = pd.DataFrame({'pred':sentences[1:]})

In [ ]:
tsd = pd.read_csv("data/tsd_trial.csv") 
tsd.spans = tsd.spans.apply(literal_eval)

toxic_offsets_all=[]
for i in range(len(df_dev.token)):
    toxic_offsets = []
    current_offset = 0
    for j in range(len(df_dev.token[i])):
        if df_temp.pred[i][j]==1:
            toxic_offsets.extend(list(range(current_offset, current_offset+len(df_dev.token[i][j]))))
        current_offset += len(df_dev.token[i][j]) + 1
    toxic_offsets_all.append(toxic_offsets)
df_predict = pd.DataFrame({'pred':toxic_offsets_all})

In [ ]:
tsd["predictions"] = df_predict.pred
tsd["f1_scores"] = tsd.apply(lambda row: f1(row.predictions, row.spans), axis=1)
print (f"F1 = {tsd.f1_scores.mean():.2f} ± {sem(tsd.f1_scores):.2f}")

---

# System 4 (with spacy)

In [ ]:
import nltk
from nltk.corpus import words
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
from nltk.metrics.distance  import edit_distance

In [ ]:
# Datasets Loading

# training data
dataset = pd.read_csv("data/tsd_train.csv")
# development and validation data
tsd = pd.read_csv("data/tsd_trial.csv")

dataset.spans = dataset.spans.apply(literal_eval)
tsd.spans = tsd.spans.apply(literal_eval)

**Toxic words from lexicons**

In [ ]:
lex_1 = pd.read_csv("lexicon/base_list_bad_words.csv")

lex_2 = pd.read_csv("lexicon/swearWords.csv").T
lex_2 = lex_2.reset_index()
lex_2 = lex_2.rename(columns={"index": "bad_word"})

lex_3 = pd.read_csv("lexicon/Terms-to-Block.csv")

lex_4 = pd.read_table("lexicon/bad-words.txt", delim_whitespace=True)

In [ ]:
#####################
lex_1 = lex_1.apply(lambda x: x.str.replace(';',''))
correct_spellings_1 = set(lex_1.bad_word.tolist())

lex_1.head()

In [ ]:
#####################
correct_spellings_2 = set(lex_2.bad_word.tolist())
lex_2.head()

In [ ]:
#####################
lex_3 = lex_3.apply(lambda x: x.str.replace(',',''))
correct_spellings_3 = set(lex_3.bad_word.tolist())
lex_3.head()

In [ ]:
#####################
correct_spellings_4 = set(lex_4.bad_word.tolist())
lex_4.head()

In [ ]:
##################### with all lexicons together
lexicon_toxic = pd.read_pickle('lexicon_toxic')
lexicon_toxic
#####################

In [ ]:
correct_spellings = set(lexicon_toxic.bad_word.tolist())

**Start algorithm**

In [ ]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Tranform from a format (character list) to other format (word list)
def spans_to_ents(doc, toxic_spans):
    started = False
    left, right, ents = 0, 0, []
    for token in doc:
        if token.pos_ == 'SPACE': continue
        # if the token is in the spans return just the starting-ending offsets
        if toxic_spans.intersection(set(range(token.idx, token.idx + len(token.text)))):
            if not started:        
                left, started = token.idx, True
            right = token.idx + len(token.text)
        # this is activated when "started" is True and we moved on a non-toxic word,
        # so, just return the saved left/right offsets.
        elif started:
            ents.append((left, right, 'TOXIC'))
            started = False
    if started: 
        ents.append((left, right, 'TOXIC'))
    return ents

In [ ]:
idx=2342
print(dataset.iloc[idx].text, dataset.iloc[idx].spans)
spans_to_ents(nlp(dataset.iloc[idx].text), set(dataset.iloc[idx].spans))

In [ ]:
TRAIN_DATA = []
#create the new dataset,which we use as training dataset
for _, row in tqdm(dataset.iterrows()):
    doc = nlp(row.text)
    ents = spans_to_ents(doc, set(row.spans))
    TRAIN_DATA.append((doc.text, {'entities': ents}))

In [ ]:
TRAIN_DATA

In [ ]:
nlp = spacy.blank('en')

# We have name regognition task, so we need
# new NER pipe with TOXIC entities
ner = nlp.create_pipe("ner")
ner.add_label('TOXIC')

# add the NER pipe as last component 
nlp.add_pipe(ner, last=True)

In [ ]:
# exclude some pipes from being updated during training
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
from IPython.display import SVG, display
def show_svg():
    display(SVG(url='https://spacy.io/training-73950e71e6b59678754a87d6cf1481f9.svg'))
    
show_svg()

In [ ]:
import random 
with nlp.disable_pipes(*unaffected_pipes):
    
    #start to train
    nlp.begin_training()
    
    # 30 epochs
    for iteration in tqdm(range(34)):
        
        #Shuffle the train dataset.
        random.shuffle(TRAIN_DATA)
        
        losses = {}
        
        #split in to batches
        batches = spacy.util.minibatch(TRAIN_DATA)
        
        #for each batch, update the annotations
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, drop=0.5,  losses=losses)
            
        print("Losses", losses)

#### Result of develop set

In [ ]:
for _, row in tqdm(tsd.iterrows()):
    spacy.displacy.render(nlp(row.text), style='ent', jupyter=True)

In [ ]:
for _, row in tqdm(tsd.iterrows()):
    spacy.displacy.render(nlp(row.text), style='ent', jupyter=True)

In [ ]:
toxic_offsets_all=[]
for idx in tqdm(range(len(tsd.spans))):
    toxic_offsets=[]
    for token in nlp(tsd.text[idx]):
        if token.ent_type_=='TOXIC':
            temp = np.arange(token.idx, token.idx + len(token)).tolist()
            toxic_offsets.extend(temp)
            
            
        else:
            token_str= str(token.text).lower()
            if token_str in correct_spellings_1:
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
                print('lex_1---->'+str(token_str))
            elif token_str in correct_spellings_2:
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
                print('lex_2---->'+str(token_str))
            elif token_str in correct_spellings_3:
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
                print('lex_3---->'+str(token_str))
            elif token_str=='g&m':
                trmp = token_str
                # tranform money to TAG (eg 34$ )
                token_str= re.sub('g&m', 'gm', token_str)
            elif token_str=='fukc':
                print(token_str)
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
            elif token_str=='fukcer':
                print(token_str)
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
            elif token_str=='sh__' or token_str=='sh-t' or token_str=='muslims':
                print(token_str)
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
            else:
                trmp = token_str
                # tranform percentage to TAG (eg 34% )
                token_str= re.sub(r'\d+[%]', 'NUM', token_str)
    
                # tranform money to TAG (eg 34$ )
                token_str= re.sub(r'\d+[$]', 'MONEY', token_str)
    
                # tranform money to TAG (eg $34)
                token_str= re.sub(r'[$]\d+', 'MONEY', token_str)
    
                # tranform money to TAG (eg $34)
                token_str= re.sub(r'[$]\w+', 'TOXIC', token_str)
            
                # change hide toxic words (e.g f@@k = f**k)
                token_str= re.sub(r"(\w+[*+$#@&%!]+\w+)", 'TOXIC', token_str)
    
                # change hide toxic words (e.g f***)
                token_str= re.sub(r"(\w+[*$#@&%])", 'TOXIC', token_str)
                
                if token_str=='TOXIC':
                    print(trmp)
                    temp = np.arange(token.idx, token.idx + len(token)).tolist()
                    toxic_offsets.extend(temp)
    toxic_offsets_all.append(toxic_offsets)

**Predict**

In [ ]:
df_predict = pd.DataFrame({'pred':toxic_offsets_all})
df_predict

In [ ]:
def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [ ]:
tsd["predictions"] = df_predict.pred
tsd["f1_scores"] = tsd.apply(lambda row: f1(row.predictions, row.spans), axis=1)

In [ ]:
from scipy.stats import sem
print (f"F1 = {tsd.f1_scores.mean():.2f} ± {sem(tsd.f1_scores):.2f}")

#### Result of test set

In [ ]:
# test data
dataset_test = pd.read_csv("data/tsd_test.csv")

In [ ]:
for _, row in tqdm(dataset_test.iterrows()):
    spacy.displacy.render(nlp(row.text), style='ent', jupyter=True)

In [ ]:
toxic_offsets_all=[]
for idx in tqdm(range(len(dataset_test.text))):
    toxic_offsets=[]
    for token in nlp(dataset_test.text[idx]):
        if token.ent_type_=='TOXIC':
            temp = np.arange(token.idx, token.idx + len(token)).tolist()
            toxic_offsets.extend(temp)
            
        else:
            token_str= str(token.text).lower()
            if token_str in correct_spellings_1:
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
                print('lex_1---->'+str(token_str))
            elif token_str in correct_spellings_2:
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
                print('lex_2---->'+str(token_str))
            elif token_str in correct_spellings_3:
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
                print('lex_3---->'+str(token_str))
            elif token_str=='g&m':
                trmp = token_str
                # tranform money to TAG (eg 34$ )
                token_str= re.sub('g&m', 'gm', token_str)
            elif token_str=='fukc':
                print(token_str)
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
            elif token_str=='fukcer':
                print(token_str)
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
            elif token_str=='sh__' or token_str=='sh-t' or token_str=='muslims':
                print(token_str)
                temp = np.arange(token.idx, token.idx + len(token)).tolist()
                toxic_offsets.extend(temp)
            else:
                trmp = token_str
                # tranform percentage to TAG (eg 34% )
                token_str= re.sub(r'\d+[%]', 'NUM', token_str)
    
                # tranform money to TAG (eg 34$ )
                token_str= re.sub(r'\d+[$]', 'MONEY', token_str)
    
                # tranform money to TAG (eg $34)
                token_str= re.sub(r'[$]\d+', 'MONEY', token_str)
    
                # tranform money to TAG (eg $34)
                token_str= re.sub(r'[$]\w+', 'TOXIC', token_str)
            
                # change hide toxic words (e.g f@@k = f**k)
                token_str= re.sub(r"(\w+[*+$#@&%!]+\w+)", 'TOXIC', token_str)
    
                # change hide toxic words (e.g f***)
                token_str= re.sub(r"(\w+[*$#@&%])", 'TOXIC', token_str)
                
                if token_str=='TOXIC':
                    print(trmp)
                    temp = np.arange(token.idx, token.idx + len(token)).tolist()
                    toxic_offsets.extend(temp)
    toxic_offsets_all.append(toxic_offsets)

In [ ]:
df_predict = pd.DataFrame({'pred':toxic_offsets_all})
df_predict

In [ ]:
predictions = df_predict.pred.to_list()
ids = df_predict.index.to_list()

# write in a prediction file named "spans-pred.txt"
with open("spans-pred.txt", "w") as out:
      for uid, text_scores in zip(ids, predictions):
            out.write(f"{str(uid)}\t{str(text_scores)}\n")

In [ ]:
! zip -r random_predictions_spacy.zip ./spans-pred.*